# Geospatial data analysis on the Planetary Computer

This section covers various geospatial data analysis topics.

In [ ]:
import pystac_client
import planetary_computer
import stackstac
import rasterio
import xrspatial.multispectral
import matplotlib.pyplot as plt
import numpy as np

from distributed import Client

client = Client()
client

## Coregistration

When you're working with data from multiple sources, you'll want to coregister, or align, your data. This ensures that a value at position `(x, y)` from one dataset represents the same area as the value at position `(x, y)` from some other dataset.

This example loads some data near Yazoo National Wildlife Refuge ([explorer](https://planetarycomputer.microsoft.com/explore?c=-90.9831%2C33.0868&z=12.20&d=sentinel-2-l2a&m=Most+recent+%28low+cloud%29&r=Natural+color)). We'll search for some STAC items over this area from Sentinel-2 Level 2-A.

In [ ]:
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [-91.92, 33.43],
            [-90.74, 33.41],
            [-90.76, 32.42],
            [-91.93, 32.44],
            [-91.92, 33.43],
        ]
    ],
}
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)

sentinel_search = catalog.search(
    intersects=area_of_interest,
    datetime="2020-09-07/2020-09-08",
    collections=["sentinel-2-l2a"],
)

sentinel_item = next(sentinel_search.get_items())  # select the first item
sentinel_item = planetary_computer.sign(sentinel_item)

And from Landsat 8 Collection 2 Level 2.

In [ ]:
search = catalog.search(
    collections=["landsat-8-c2-l2"],
    intersects=sentinel_item.geometry,
    datetime="2020-09-01/2020-09-30",
    query={
        "eo:cloud_cover": {
            "lt": 10,
        }
    },
)

landsat_items = [planetary_computer.sign(item).to_dict() for item in search.get_items()]

Now we'll load these two datasets with xarray and stackstac.

In [ ]:
sentinel_data = (
    (
        stackstac.stack(
            sentinel_item,
            resolution=100,
            assets=["B02", "B03", "B04", "B08"],  # blue, green, red, nir
        )
        .where(lambda x: x > 0)  # sentinel-2 uses 0 as nodata
        .assign_coords(band=lambda x: x.common_name.rename("band"))  # use common names
    )
    .isel(time=0)
    .rename("sentinel")
    .persist()
)
sentinel_data

Now we'll load the Landsat data, but we'll make sure to coregister it with the Sentinel data. This will include

- resampling the Landsat item to match the resolution of the Sentinel data
- reprojecting the Landsat items to the CRS of the Sentinel data
- cropping the Landsat data to the bounds of the Sentinel data

In [ ]:
landsat_data = (
    (
        stackstac.stack(
            landsat_items,
            resolution=sentinel_data.resolution,  # resample to Sentinel data resolution
            epsg=sentinel_data.spec.epsg,  # reporoject to CRS of Sentinel data
            bounds=sentinel_data.spec.bounds,  # set bounds to match Sentinel data
            assets=["SR_B2", "SR_B3", "SR_B4", "SR_B5"],  # blue, green, red, nir
        )
        .where(lambda x: x > 0)  # landsat-8 uses 0 as nodata
        .assign_coords(band=sentinel_data.band.data)
    )
    .pipe(stackstac.mosaic)
    .rename("landsat")
    .persist()
)

landsat_data

Now the data are coregistered. The coordinates match, so the values at each coordinate represent the same area on earth.

In [ ]:
(landsat_data.x.data == sentinel_data.x.data).all()

In [ ]:
(landsat_data.y.data == sentinel_data.y.data).all()

Which means we can safely combine them into a single `xarray.Dataset`.

In [ ]:
import xarray as xr

ds = xr.combine_by_coords([sentinel_data, landsat_data], join="exact", compat="override")
ds

With this coregistered dataset, we can do operations using data from each source. For example, you could compute NDVI using a red band from Landsat 8 and a near-infrared band from Sentinel 2.

In [ ]:
nir = ds.sentinel.sel(band="nir")
red = ds.landsat.sel(band="red")

ndvi = (nir - red) / (nir + red)

In [ ]:
m = stackstac.show(ndvi, range=[-0.85, -0.25])
m.scroll_wheel_zoom = True
m

To recap

- We used `pystac-client` to search the STAC API for Sentinel-2 and Landsat 8 scenes matching our criteria
- We used `stackstac` to create DataArrays from both sets of STAC items, taking care to reproject and coregister the Landsat data to the same grid as the Sentinel data
- We used standard xarray operations to combine the two DataArrays and compute NDVI

## Surface tools

Surface tools help you visualize and analyze elevation data. In this example, we'll look at data around Grand Teton National Park using data from the NASADEM digital elevation model ([explorer](https://planetarycomputer.microsoft.com/explore?c=-109.8162%2C43.5419&z=7.43&d=nasadem&r=Elevation+%28terrain%29&m=Most+recent)).

In [ ]:
import planetary_computer as pc
import pystac_client
import stackstac

from datashader import Canvas
from datashader.colors import Elevation
from datashader.transfer_functions import shade
from datashader.transfer_functions import stack
from xrspatial.utils import height_implied_by_aspect_ratio

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/"
)
point = {"type": "Point", "coordinates": [-110.6818, 43.7904]}

search = catalog.search(collections=["nasadem"], intersects=point, limit=1)
nasadem_item = next(search.get_items())

nasadem_item

In [ ]:
elevation_raster = stackstac.stack(
    [pc.sign(nasadem_item).to_dict()],
    epsg=6933,
    resampling=rasterio.enums.Resampling.bilinear,
    chunksize=2048,
).squeeze()

elevation_raster

In [ ]:
# get full extent of raster
full_extent = (
    elevation_raster.coords["x"].min().item(),
    elevation_raster.coords["y"].min().item(),
    elevation_raster.coords["x"].max().item(),
    elevation_raster.coords["y"].max().item(),
)

# get ranges
x_range = (full_extent[0], full_extent[2])
y_range = (full_extent[1], full_extent[3])

# set width and height
W = 300
H = height_implied_by_aspect_ratio(W, x_range, y_range)

cvs = Canvas(plot_width=W, plot_height=H, x_range=x_range, y_range=y_range)

In [ ]:
elevation_small = cvs.raster(
    elevation_raster,
)
elevation_img = shade(elevation_small, cmap=Elevation, how="linear")
elevation_img

In [ ]:
from xrspatial import hillshade

hillshade_raster = hillshade(elevation_raster)
hillshade_raster

In [ ]:
hillshade_img = shade(
    cvs.raster(hillshade_raster),
    cmap=["#333333", "#C7C7C7"],
    alpha=200,
)

hillshade_img

In [ ]:
terrain_img = shade(elevation_small, cmap=Elevation, alpha=128, how="linear")
stack(hillshade_img, terrain_img)

In [ ]:
from xrspatial import aspect
from datashader.colors import viridis

aspect_raster = aspect(elevation_raster)
aspect_raster

In [ ]:
aspect_img = shade(cvs.raster(aspect_raster), cmap=viridis)
aspect_img

In [ ]:
aspect_img = shade(cvs.raster(aspect_raster), cmap=viridis, alpha=128)
stack(elevation_img, hillshade_img, aspect_img)

xarray-spatial implements various other surface analysis tools like `curvature` and `slope`.

## Proximity

Proximity tools help you measure the distance between points. In this example, we'll use the JRC Global Surface Water dataset to measure the distance to the nearest water for a region of the Amazon rainforest ([Explorer](https://planetarycomputer.microsoft.com/explore?c=-52.5652%2C-7.4759&z=4.00&d=jrc-gsw&m=Most+recent&r=Water+occurrence]))

In [ ]:
from xrspatial import proximity
from matplotlib.colors import Normalize, ListedColormap

bounds = [-57.151965, -2.530125, -55.710724, -1.179033]

jrc = catalog.search(collections=["jrc-gsw"], bbox=bounds)

items = list(jrc.get_items())
print(f"Returned {len(items)} Items")

In [ ]:
item = items[0]

assets_of_interest = ["extent", "seasonality", "transitions"]

data = (
    stackstac.stack(
        [pc.sign(item).to_dict()],
        assets=assets_of_interest,
        bounds=bounds,
        chunksize=3000,
    )
    .isel(time=0)
    .persist()
)

data

In [ ]:
cmaps = {}

for asset_key in assets_of_interest:
    asset = item.assets[asset_key]
    with rasterio.open(pc.sign(item.assets[asset_key].href)) as src:
        colormap_def = src.colormap(1)  # get metadata colormap for band 1
        colormap = [
            np.array(colormap_def[i]) / 256 for i in range(256)
        ]  # transform to matplotlib color format
    cmaps[asset_key] = ListedColormap(colormap)

In [ ]:
norm = Normalize(0, 255)

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 15))

for i, asset_key in enumerate(assets_of_interest):
    ax[i].imshow(
        data.sel(band=asset_key),
        norm=norm,
        cmap=cmaps[asset_key],
    )
    ax[i].set_title(asset_key, fontdict={"fontsize": 15})
    ax[i].set_axis_off()

We'll use xarray-spatial's `proximity` function. By default, this computes the distance (euclidean by default) from each point in the dataset to the nearest *non-zero* point. That works well with the JRC dataset, which sets points that aren't covered by water to 0.

In [ ]:
extent_data = data.sel(band="extent").compute()
extent_data

In [ ]:
extent_proximity_default = proximity(extent_data)
extent_proximity_default.name = """
    Water Extent proximity distance
    (Euclidean max_distance=infinity)
"""

In [ ]:
extent_proximity_default

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

extent_proximity_default.plot.imshow(cmap="Greens_r", add_colorbar=False, ax=ax)
extent_data.plot.imshow(
    norm=norm, cmap=cmaps["extent"], add_colorbar=False, alpha=0.5, ax=ax
)
ax.set_axis_off()
ax.set(title=extent_proximity_default.name);

There are various other ways to compute proximity, for example the direction to the nearest water.

## Zonal statistics

Zonal statistics help you better understand data from one source by analyzing it for different zones defined by another source.

* One dataset gives the *zones* (typically discrete integer codes)
* A second dataset gives the *values*, which are aggregated per zone

In [ ]:
bounds = (-98.00080760573508, 32.99921674609716, -96.9991860639418, 34.000729644613706)

landcover_search = catalog.search(collections=["io-lulc"], bbox=bounds)
landcover_items = list(landcover_search.get_items())
signed_items = [pc.sign(item).to_dict() for item in landcover_items]

landcover_data = (
    stackstac.stack(
        signed_items,
        epsg=3857,
        bounds_latlon=bounds,
        dtype="int8",
        fill_value=0,
        chunksize=2048,
        resolution=100,
    )
    .pipe(stackstac.mosaic)
    .squeeze()
    .rename("Landcover")
    .persist()
)
landcover_data

In [ ]:
landcover_labels = dict(
    enumerate(landcover_data.coords["label:classes"].item()["classes"])
)
landcover_labels

In [ ]:
import numpy as np
from matplotlib.colors import ListedColormap

class_count = len(landcover_labels)

with rasterio.open(signed_items[0]["assets"]["data"]["href"]) as src:
    landcover_colormap_def = src.colormap(1)  # get metadata colormap for band 1
    landcover_colormap = [
        np.array(landcover_colormap_def[i]) / 255 for i in range(class_count)
    ]

landcover_cmap = ListedColormap(landcover_colormap)
landcover_cmap

In [ ]:
sentinel_search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bounds,
    datetime="2020-07-01/2020-07-30",
    query={
        "eo:cloud_cover": {
            "lt": 10,
        }
    },
)

sentinel_items = [pc.sign(item).to_dict() for item in sentinel_search.get_items()]

sentinel_data = (
    (
        stackstac.stack(
            sentinel_items,
            resolution=landcover_data.resolution,
            epsg=landcover_data.spec.epsg,
            bounds=landcover_data.spec.bounds,
            assets=["B02", "B03", "B04", "B08"],  # blue, green, red, nir
            chunksize=2048,
        )
        .assign_coords(band=lambda x: x.common_name.rename("band"))  # use common names
        .where(lambda x: x > 0, other=np.nan)  # Sentinels uses 0 as nodata
    )
    .median(dim="time", keep_attrs=True)
    .persist()
)
sentinel_data

In [ ]:
sentinel_img = xrspatial.multispectral.true_color(
    sentinel_data.sel(band="red"),
    sentinel_data.sel(band="green"),
    sentinel_data.sel(band="blue"),
    c=30,
    th=0.075,
    name="True color (Sentinel)",
)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(18, 8))

landcover_data.plot.imshow(ax=ax1, cmap=landcover_cmap, add_colorbar=False)
sentinel_img.plot.imshow(ax=ax2)
plt.tight_layout()

In [ ]:
red, nir = sentinel_data.sel(band=["red", "nir"])
ndvi = ((nir - red) / (nir + red)).persist()
ndvi

So `landcover_data` is our *zones* raster, and `ndvi` is our `values` raster.

In [ ]:
import dask
from xrspatial import zonal_stats

landcover_data, ndvi = dask.compute(landcover_data, ndvi)

quantile_stats = zonal_stats(
    zones=landcover_data,
    values=ndvi,
    stats_funcs=["mean", "max", "min", "count"],
)
quantile_stats

In [ ]:
styler = (
    quantile_stats.set_index("zone")
    .rename(landcover_labels)
    .round(2)
    .style.format(precision=2)
    .background_gradient()
)
styler

In [ ]:
quantile_stats.set_index("zone").rename(landcover_labels)["mean"].sort_values().plot.barh(width=0.9);